In [156]:
from collections import defaultdict

In [157]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [158]:
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i+1] = p

In [44]:
price[20]

0

In [159]:
def r(n):
    return(max([price[n]] + [r(i) + r(n-i) for i in range(1,n)]))

In [46]:
r(10)

30

In [162]:
solution = {}

In [160]:
from functools import wraps

In [161]:
def memo(f):
    memo.already_computed = {}
    @wraps(f) # decorator after python 2.7 and python 3.6
    def _wrap(arg):
        result = None
        
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        
        return result
    return _wrap

In [163]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue
    """
    max_price, max_split = max([(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1,n)]\
                              , key = lambda x: x[0])
    
    solution[n] = (n - max_split, max_split)
    
    return max_price

In [39]:
r(20)

60

In [40]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

# Parse solution

In [49]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0:
        return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [50]:
r(321)

1020

In [52]:
'🙅'.join(map(str,parse_solution(321)))

'11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅11🙅2'

## Edit distance

In [181]:
solution = {}

In [164]:
from functools import lru_cache

In [187]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if len(string1)==0: return len(string2)
    if len(string2)==0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2))
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB: {} = {}'\
                        .format(tail_s1, tail_s2))
        
    candidates.append(both_forward)    
    min_distance, operation = min(candidates, key = lambda x: x[0])   
    solution[(string1, string2)] = operation
    return min_distance

In [188]:
edit_distance('ABCDE', 'ABBCEF')

3

In [189]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABB'): 'ADD B',
 ('A', 'ABBC'): 'ADD C',
 ('A', 'ABBCE'): 'ADD E',
 ('A', 'ABBCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABB'): 'ADD B',
 ('AB', 'ABBC'): 'ADD C',
 ('AB', 'ABBCE'): 'ADD E',
 ('AB', 'ABBCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABB'): 'SUB: C = B',
 ('ABC', 'ABBC'): '',
 ('ABC', 'ABBCE'): 'ADD E',
 ('ABC', 'ABBCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABB'): 'DEL D',
 ('ABCD', 'ABBC'): 'DEL D',
 ('ABCD', 'ABBCE'): 'SUB: D = E',
 ('ABCD', 'ABBCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABB'): 'DEL E',
 ('ABCDE', 'ABBC'): 'DEL E',
 ('ABCDE', 'ABBCE'): '',
 ('ABCDE', 'ABBCEF'): 'ADD F'}

## homework-parse_edit_distance_solution

In [178]:
def parse_edit_distance_solution(string1, string2):
    if len(string1) > 0:
        temp = solution[(string1, string2)]
        parsed_solution.append(temp)

        if temp.startswith('ADD'):
            parse_edit_distance_solution(string1, string2[:-1])
        elif temp.startswith('DEL'):
            parse_edit_distance_solution(string1[:-1], string2)
        else:
            parse_edit_distance_solution(string1[:-1], string2[:-1])

        return [i for i in parsed_solution if i != '']

In [190]:
parsed_solution = []

In [191]:
edit_solution = parse_edit_distance_solution('ABCDE', 'ABBCEF')

In [192]:
edit_solution

['ADD F', 'DEL D', 'ADD B']

In [193]:
parsed_solution = []
clear_solution = parse_edit_distance_solution('ABCDE', 'ABBCEF')
print('-->'.join(['ABCDE'] + clear_solution + ['ABBCEF']))

ABCDE-->ADD F-->DEL D-->ADD B-->ABBCEF


## Pinyin Auto Correction Problem


In [194]:
path='D:/Github/AIClass/datasource_lecutre/datasource/'

In [196]:
chinese_dataset = path +  'article_9k.txt'
CHINESE_CHARACTERS = open(chinese_dataset, 'r', encoding='UTF-8').read()
print(CHINESE_CHARACTERS[:8])

此外自本周6月1


In [197]:
import pinyin

In [198]:
pinyin.get('你好', format='strip', delimiter=' ')

'ni hao'

In [199]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [200]:
CHINESE_PINYIN_COPYS = chinese_to_pinyin(CHINESE_CHARACTERS)

In [69]:
len(CHINESE_PINYIN_COPYS)

129433034

In [201]:
import re
def tokens(text):
    return re.findall('[a-z]+', text.lower())
print(tokens(CHINESE_PINYIN_COPYS[:100]))

['ci', 'wai', 'zi', 'ben', 'zhou', 'yue', 'ri', 'qi', 'chu', 'xiao', 'mi', 'shou', 'ji', 'deng', 'kuan', 'ji', 'xing', 'wai', 'qi', 'yu', 'ji', 'xing', 'yi']


In [202]:
from collections import Counter, defaultdict

PINYIN_COUNT = Counter(tokens(CHINESE_PINYIN_COPYS))

In [73]:
def correct(word):
    candidates = (known(edits0(word)) or known(edits1(word)) or known(edits2(word)) or [word])
    
    return max(candidates, key = PINYIN_COUNT.get)
    

In [74]:
def known(words):
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [75]:
def edits1(word):
    pairs = splits(word)
    deletes = [a + b[1:] for (a, b) in pairs if b]
    transposes = [a + b[1] + b[0] + b[2:] for (a, b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a, b) in pairs for c in alphabets if b]
    inserts = [a+c+b for (a, b) in pairs for c in alphabets]
    
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) for i in range(len(word) + 1)]

alphabets = 'abcdefghijklmnopqrstuvwxyz'
    

In [76]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [77]:
[i for i in edits1('shen') if i in PINYIN_COUNT]

['hen', 'zhen', 'chen', 'she', 'sheng', 'sen', 'shan', 'shen', 'shun']

In [78]:
correct('pign')

'ping'

In [79]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [80]:
correct_sequence_pinyin('zhe shi yi ge ce shi')

'zhe shi yi ge ce shi'

In [81]:
correct_sequence_pinyin('wo xiang shang qinng hau da xue')

'wo xiang shang qing hua da xue'

In [82]:
words_count= Counter(tokens(CHINESE_PINYIN_COPYS))

In [84]:
tokens_cut=tokens(CHINESE_PINYIN_COPYS)

In [85]:
tokens_cut[:5]

['ci', 'wai', 'zi', 'ben', 'zhou']

In [83]:
words_count.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('n', 691114),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441)]

In [86]:
_2_gram_words=[
    tokens_cut[i]+tokens_cut[i+1] for i in range(len(tokens_cut)-1)
]

In [88]:
_2_gram_words[:10]

['ciwai',
 'waizi',
 'ziben',
 'benzhou',
 'zhouyue',
 'yueri',
 'riqi',
 'qichu',
 'chuxiao',
 'xiaomi']

In [89]:
_2_gram_word_counts=Counter(_2_gram_words)

In [90]:
_2_gram_word_counts.most_common(10)

[('yueri', 165274),
 ('xinhua', 151823),
 ('huashe', 145795),
 ('nn', 136525),
 ('nxin', 89783),
 ('zhongguo', 87864),
 ('waidai', 83330),
 ('nianyue', 77695),
 ('jizhe', 65213),
 ('nwai', 62999)]

In [91]:
def get_garm_count(word,wc):
    if word in wc:return wc[word]
    else:
        return wc.most_common()[-1][-1]

In [ ]:
def two_gram_model(sentence):
    tokens=cut(sentence)
    
    probability=1
    
    for i in range(len(tokens)-1):
        word=tokens[i]
        next_word=tokens[i+1]
        
        _two_gram_c=get_garm_count(word+next_word,_2_gram_word_counts)
        _one_gram_c=get_garm_count(next_word,words_count)
        pro= _two_gram_c/_one_gram_c
        
        probability*=pro
    
    return probability

##  homework:one-gram model

In [203]:
def p_dist(counter):
    N = sum(counter.values())
    return lambda x: (counter[x])/N

prob = p_dist(PINYIN_COUNT)

In [204]:
#one-gram model
def p_one_gram(text):
    '''
    P(w1…wn)=P(w1)×P(w2)×P(w3)…×…P(wn)
    '''
    product = 1
    for w in text:
            product *= prob(w)
    return product

In [205]:
p_one_gram('qinghua'.split())

0.0

In [206]:
p_one_gram('qing hua'.split())

2.9837508084361854e-05

In [207]:
def split_text(text, start=0, L=20):
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

In [209]:
@lru_cache(maxsize=2**10)
def segment(text):
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in split_text(text,1))
        return max(candidates, key=p_one_gram)

In [212]:
segment('woxiangshagqingnhuuadaxue')

['wo', 'xiang', 'sha', 'g', 'qing', 'n', 'hu', 'u', 'a', 'da', 'xue']

In [213]:
segment('woxiangshangqinnghuadaxue')

['wo', 'xiang', 'shang', 'qin', 'n', 'g', 'hua', 'da', 'xue']

In [223]:
def correct_segment(text, seg_fn):
    word=''
    seg = seg_fn(text)

    for i, w in enumerate(seg):
       # print("i",i)
       # print("w",w)
        l = len(w)
        if i == 0: 
            wl = 0
            lastl = 0
        if (l==1 or lastl==1) and (wl+l)<=5:
            word += w
            wl += l
        else:
            word = word + ' ' + w
            wl = l
        lastl = l
    return correct_sequence_pinyin(word)

In [224]:
correct_segment('woxiangshangqinnghuadaxue',segment)

'wo xiang shang qing hua da xue'

In [220]:
correct_segment('nizhongwuuchifanlema',segment)

'ni zhong wu chi fan le ma'

总结：尝试了 1-gram model，效果还可以；但没想出来 2-gram model 方法。
